In [1]:
%pip install google-auth google-auth-oauthlib google-auth-httplib2 gspread pandas

StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 7, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 30.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, trunc, last_day, concat, lit, date_format, from_utc_timestamp
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, trunc, last_day, date_format
from datetime import datetime
from pyspark.sql.functions import lit, from_utc_timestamp
from dateutil.relativedelta import relativedelta
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
from pyspark.sql import SparkSession
import json
from pyspark.sql import Row
import requests
from pyspark.sql.functions import col, last_day, concat, lit, to_timestamp
from pyspark.sql.functions import col, trunc, concat, lit, to_timestamp
from pyspark.sql.functions import to_timestamp, trunc, last_day, concat, lit


StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 9, Finished, Available, Finished)

In [3]:
# Path to stored service account JSON in Fabric
file_path = "./builtin/service_account.json"




# Load JSON file into a dictionary
with open(file_path, "r") as file:
    json_dict = json.load(file)

# Convert dictionary to string
json_str = json.dumps(json_dict)

StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 10, Finished, Available, Finished)

In [4]:
def call_api_with_retries(api_func, *args, max_retries=5, base_wait=2):
    retries = 0
    while retries < max_retries:
        response = api_func(*args)  # Call the API function
        
        if isinstance(response, requests.Response) and response.status_code == 429:
            # Get retry wait time from header or use exponential backoff
            retry_after = int(response.headers.get("Retry-After", base_wait * (2 ** retries)))
            print(f"Rate limited. Retrying in {retry_after} seconds...")
            time.sleep(retry_after)
            retries += 1
        else:
            return response  # Return response if successful or not a 429 error

    raise Exception("Max retries reached due to repeated 429 errors.")

StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 11, Finished, Available, Finished)

In [5]:
def process_data_for_monthly_dashboard(filtered_df, channel, client, authorization):

    """Processes data for the monthly dashboard by fetching API metrics.

    Args:
        filtered_df (pd.DataFrame): User data to process.
        channel (str): The channel (e.g., 'email').
        client (str): Client name.
        authorization (str): API authorization token.

    Returns:
        pd.DataFrame: Processed data including FRT, One-Touch, Closed Tickets, and Messages Sent.
    """

    results = []


    count = 0  

    
    
    for _, row in filtered_df.iterrows():
        userid = int(row["ID"]) 
        Name = row["Name"]
        Email = row["Email"]
        
        api_result_closed_tickets = 0
        api_result_FRT = 0
        api_result_closed_tickets = 0

        for i in range(1,20):
            start_time, end_time = get_month_start_end(i)
            print(f"start_time, {start_time}")

            try:
                api_result_FRT = call_api_with_retries(call_api_for_FRT, userid, start_time, end_time, channel, client, authorization)
                
                api_result_one_touch = call_api_with_retries(call_api_for_one_touch, userid, start_time, end_time, channel, client, authorization)
                
                api_result_closed_tickets = call_api_with_retries(call_api_for_closed_tickets, userid, start_time, end_time, channel, client, authorization)
                
                api_result_messages_sent = call_api_with_retries(call_api_for_messages_sent, userid, start_time, end_time, channel, client, authorization)

                
                ART = call_api_with_retries(call_api_median_resolution, userid, start_time, end_time, channel, client, authorization)
                
                Cust_sat_sent, Cust_sat_rate, Cust_sat_rating = call_api_with_retries(call_api_for_satisfaction_survey, userid, start_time, end_time, channel, client, authorization)

                
                #print(f"custmer sat: {Customer_Sat}")
                #print(f"Name: {Name}, Channel: {channel}, api_result_FRT: {api_result_FRT}, ART: {ART}, api_result_one_touch: {api_result_one_touch}, api_result_closed_tickets: {api_result_closed_tickets}")

                #if any(x not in (None) for x in [api_result_FRT, api_result_one_touch, api_result_closed_tickets, api_result_messages_sent, ART]):
                #count = count+1
                #print(f"count: {count}")

                api_result_FRT = int(api_result_FRT) if api_result_FRT else 0
                api_result_one_touch = int(api_result_one_touch) if api_result_one_touch else 0
                api_result_closed_tickets = int(api_result_closed_tickets) if api_result_closed_tickets else 0
                api_result_messages_sent = int(api_result_messages_sent) if api_result_messages_sent else 0
                Cust_sat_sent = int(Cust_sat_sent) if Cust_sat_sent else 0
                Cust_sat_rate = int(Cust_sat_rate) if Cust_sat_rate else 0
                Cust_sat_rating = int(Cust_sat_rating) if Cust_sat_rating else 0

                if api_result_FRT and api_result_one_touch and api_result_closed_tickets and api_result_messages_sent and ART and Cust_sat_sent and Cust_sat_rate and Cust_sat_rating:

                    
                    results.append({
                            "ID": userid,
                            "Name": Name,
                            "Email": Email,
                            "FRT": api_result_FRT,
                            "One_Touch_in_Percentage": api_result_one_touch,
                            "Ticket": api_result_closed_tickets,
                            "Messages": api_result_messages_sent,
                            "Customer_Satisfaction-Messages_Sent": Cust_sat_sent,
                            "Customer_Satisfaction-Response_Rate_in_Percentage": Cust_sat_rate,
                            "Customer_Satisfaction-Rating": Cust_sat_rating,
                            "Channel": channel,
                            "Start_date": start_time,
                            "end_date": end_time, 
                            "ART": ART,
                            "Client": client,
                            "Month_start_date": start_time
                           

                        })
            except Exception as e:
                print(f"Error processing user {Name}: {e} while executing process_data_for_monthly_dashboard function")
    
   


    return results

StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 12, Finished, Available, Finished)

In [6]:

def get_month_start_end(months_back: int = 1):
    """
    Function to calculate the start and end date of the month for a past month based on the system date.

    Args:
    months_back (int, optional): Number of months to go back (default is 1 for last month).

    Returns:
    tuple: A tuple containing the start and end dates in the format ('start_time', 'end_time').
    """
    tz_offset: str = "-05:00"

    # Get today's date
    today = datetime.today()

     # Calculate the target date by subtracting months
    start_target_date = today - relativedelta(months=months_back)
    #print(start_target_date)
    
    target_date = start_target_date.replace(day=1)  # Set to the first of the target month
   

    # Get start and end of the target month
    start_date = target_date.replace(day=1, hour=0, minute=0, second=0)
    end_date = (start_date + relativedelta(months=1)) - relativedelta(seconds=1)

    # Format the output with timezone offset
    start_str = start_date.strftime(f"%Y-%m-%dT%H:%M:%S{tz_offset}")
    end_str = end_date.strftime(f"%Y-%m-%dT%H:%M:%S{tz_offset}")

    return start_str, end_str


StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 13, Finished, Available, Finished)

In [7]:
import requests
import pandas as pd
import time

def process_users(client, authorization, max_retries=5, initial_delay=5):
    """
    Fetches users from the Gorgias API while handling rate limits (429 status code) with exponential backoff.
    """
    api_url = f"https://{client}.gorgias.com/api/users"
    headers = {
        "accept": "application/json",
        "authorization": f"Basic {authorization}"
    }
    
    retry_delay = initial_delay  # Start with the initial delay
    for attempt in range(max_retries):
        response = requests.get(api_url, headers=headers)
        
        if response.status_code in [200, 201]:
            data = response.json()
            df_user = pd.DataFrame([
                {
                    "ID": user["id"],
                    "Name": f"{user['firstname']} {user['lastname']}".strip(),
                    "Email": user["email"],
                    "Role": user["role"]["name"]
                }
                for user in data.get("data", [])
            ])
            break
        
        elif response.status_code == 429:
            retry_after = int(response.headers.get("Retry-After", retry_delay))
            print(f"Rate limited. Retrying in {retry_after} seconds...")
            time.sleep(retry_after)
            retry_delay *= 2  # Exponential backoff
        
        else:
            print(f"API request failed in process_users with status code {response.status_code}")
            return pd.DataFrame(columns=["ID", "Name", "Email", "Role"])
    else:
        print("Max retries reached. Exiting.")
        return pd.DataFrame(columns=["ID", "Name", "Email", "Role"])
    
    # Filter rows where 'email' contains 'getwrrk'
    return df_user[df_user['Email'].str.contains('getwrrk', na=False)]    


StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 14, Finished, Available, Finished)

In [8]:
import requests
from pyspark.sql.functions import col, last_day, concat, lit, to_timestamp
from pyspark.sql.functions import col, trunc, concat, lit, to_timestamp
from pyspark.sql.functions import to_timestamp, trunc, last_day, concat, lit

def call_api_for_FRT(userid, start_time, end_time, channel, client, authorization):
    """Fetches the First Response Time (FRT) metric from Gorgias API."""
    api_url = f"https://{client}.gorgias.com/api/stats/median-first-response-time"
    payload = {
        "filters": {
            "period": {
                "start_datetime": start_time,
                "end_datetime": end_time
            },
            "agents": [userid],
            "channels": [channel]
        }
    }
    headers = {
        "accept": "application/json",
        "authorization": f"Basic {authorization}"
    }
    
    retries = 0
    while retries < 5:
        response = requests.post(api_url, json=payload, headers=headers)
        
        if response.status_code == 429:
            retry_after = int(response.headers.get("Retry-After", 2 ** retries))
            print(f"Rate limited. Retrying in {retry_after} seconds...")
            time.sleep(retry_after)
            retries += 1
        elif response.status_code in [200, 201]:
            data = response.json()
            #print(f"data: {data}")
            FRT = data['data']['data']['value']  # Extract the metric value

            if FRT is not None:
                FRT_Min = FRT / 3600
            else:
                FRT_Min = 0  # or None, or skip, depending on your logic
            
            return FRT_Min

        else:
            print(f"API request failed in call_api_for_FRT function: {response.status_code}")
            return None
    
    print("Max retries reached for call_api_for_FRT.")
    return None





StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 15, Finished, Available, Finished)

In [9]:
import requests
import time
import pandas as pd
from pyspark.sql.functions import to_timestamp, trunc, last_day, concat, lit

def call_api_for_closed_tickets(userid, start_time, end_time, channel, client, authorization, max_retries=5):
    api_url = f"https://{client}.gorgias.com/api/stats/total-tickets-closed"
    
    payload = {
        "filters": {
            "period": {
                "start_datetime": start_time,
                "end_datetime": end_time
            },
            "agents": [userid],
            "channels": [channel]
        }
    }
    
    headers = {
        "accept": "application/json",
        "authorization": f"Basic {authorization}"
    }
    
    retries = 0
    while retries < max_retries:
        response = requests.post(api_url, json=payload, headers=headers)
        
        if response.status_code in [200, 201]:
            try:
                data = response.json()
                flat_data = {
                    "name": data["data"]["data"]["name"],
                    "type": data["data"]["data"]["type"],
                    "value": data["data"]["data"]["value"],
                    "delta": data["data"]["data"]["delta"],
                    "start_datetime": data["meta"]["start_datetime"],
                    "end_datetime": data["meta"]["end_datetime"],
                    "previous_start_datetime": data["meta"]["previous_start_datetime"],
                    "previous_end_datetime": data["meta"]["previous_end_datetime"]
                }
                df = pd.DataFrame([flat_data])
                return df["value"][0]
            except KeyError as e:
                print(f"Unexpected response format: Missing key {e}")
                return None
        elif response.status_code == 429:
            retry_after = int(response.headers.get("Retry-After", 2))  # Fallback to 2 seconds if no header present
            print(f"Rate limit exceeded. Retrying in {retry_after} seconds...")
            time.sleep(retry_after)
            retries += 1
        else:
            print(f"API request failed: {response.status_code} - {response.text}")
            return None
    
    print("Max retries reached. API request failed.")
    return None


StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 16, Finished, Available, Finished)

In [10]:
import requests
import time
import pandas as pd
from pyspark.sql.functions import to_timestamp, trunc, last_day, concat, lit

def call_api_for_satisfaction_survey(userid, start_time, end_time, channel, client, authorization, max_retries=5):
    api_url = f"https://{client}.gorgias.com/api/stats/satisfaction-surveys"
    
    payload = {
        "filters": {
            "period": {
                "start_datetime": start_time,
                "end_datetime": end_time
            },
            "agents": [userid],
            "channels": [channel]
        }
    }
    
    headers = {
        "accept": "application/json",
        "authorization": f"Basic {authorization}"
    }
    
    retries = 0
    while retries < max_retries:
        response = requests.post(api_url, json=payload, headers=headers)

        
        
        
        if response.status_code in [200, 201]:
            try:
                data = response.json()
                




                data_info = data.get("data", {}).get("data", [])
                meta_info = data.get("meta", {})

                rows = []

                for item in data_info:
                    if isinstance(item["value"], dict):
                        for k, v in item["value"].items():
                            flattened = {
                                "name": f"{item['name']}_{k}",
                                "type": item["type"],
                                "value": v,
                                "delta": None,
                                "more_is_better": None,
                                **meta_info
                            }
                            rows.append(flattened)
                    else:
                        flattened = {**item, **meta_info}
                        rows.append(flattened)




                df = pd.DataFrame(rows)

                #print(df)

                items = data.get("data", {}).get("data", [])
                # Extract specific values
                for item in items:
                    if item["name"] == "total_sent":
                        total_sent = item["value"]
                    elif item["name"] == "response_rate":
                        response_rate = item["value"]
                    elif item["name"] == "average_rating":
                        average_rating = item["value"]

                #print(f"total sent: {total_sent}, name: {response_rate}")
                return total_sent, response_rate, average_rating
            except KeyError as e:
                print(f"Unexpected response format: Missing key {e}")
                return None
        elif response.status_code == 429:
            retry_after = int(response.headers.get("Retry-After", 2))  # Fallback to 2 seconds if no header present
            print(f"Rate limit exceeded. Retrying in {retry_after} seconds...")
            time.sleep(retry_after)
            retries += 1
        else:
            print(f"API request failed: {response.status_code} - {response.text}")
            return None
    
    print("Max retries reached. API request failed.") 
    return None


StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 17, Finished, Available, Finished)

In [11]:
import time
import requests
import pandas as pd
from pyspark.sql.functions import col, trunc, last_day, concat, lit, to_timestamp

def call_api_for_messages_sent(userid, start_time, end_time, channel, client, authorization, max_retries=5, backoff_factor=2):
    api_url = f"https://{client}.gorgias.com/api/stats/total-messages-sent"
    
    payload = {
        "filters": {
            "period": {
                "start_datetime": start_time,
                "end_datetime": end_time
            },
            "agents": [userid], 
            "channels": [channel]
        }
    }
    
    headers = {
        "accept": "application/json",
        "authorization": f"Basic {authorization}"
    }
    
    retries = 0
    while retries < max_retries:
        response = requests.post(api_url, json=payload, headers=headers)
        
        if response.status_code in [200, 201]:
            data_dict = response.json()
            flat_data = {
                "name": data_dict["data"]["data"]["name"],
                "type": data_dict["data"]["data"]["type"],
                "value": data_dict["data"]["data"]["value"],
                "delta": data_dict["data"]["data"]["delta"],
                "start_datetime": data_dict["meta"]["start_datetime"],
                "end_datetime": data_dict["meta"]["end_datetime"],
                "previous_start_datetime": data_dict["meta"]["previous_start_datetime"],
                "previous_end_datetime": data_dict["meta"]["previous_end_datetime"]
            }
            
            df = pd.DataFrame([flat_data])
            return df["value"][0]
        
        elif response.status_code == 429:
            retry_after = int(response.headers.get("Retry-After", backoff_factor ** retries))
            print(f"Rate limit exceeded. Retrying in {retry_after} seconds...")
            time.sleep(retry_after)
            retries += 1
        
        else:
            print(f"API request failed in call_api_for_messages_sent function: {response.status_code}")
            return None  
    
    print("Max retries reached. API request failed.")
    return None


StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 18, Finished, Available, Finished)

In [12]:
import requests
import time
import pandas as pd
from pyspark.sql.functions import col, trunc, concat, lit, to_timestamp, last_day

def call_api_median_resolution(userid, start_time, end_time, channel, client, authorization, max_retries=5):
    api_url = f"https://{client}.gorgias.com/api/stats/median-resolution-time"

    payload = {
        "filters": {
            "period": {
                "start_datetime": start_time,
                "end_datetime": end_time
            },
            "agents": [userid],
            "channels": [channel]
        }
    }

    headers = {
        "accept": "application/json",
        "authorization": f"Basic {authorization}"
    }

    retries = 0
    while retries < max_retries:
        response = requests.post(api_url, json=payload, headers=headers)
        
        if response.status_code in [200, 201]:  # Success
            try:
                data_dict = response.json()
                flat_data = {
                    "name": data_dict["data"]["data"]["name"],
                    "type": data_dict["data"]["data"]["type"],
                    "value": data_dict["data"]["data"]["value"],
                    "delta": data_dict["data"]["data"]["delta"],
                    "start_datetime": data_dict["meta"]["start_datetime"],
                    "end_datetime": data_dict["meta"]["end_datetime"],
                    "previous_start_datetime": data_dict["meta"]["previous_start_datetime"],
                    "previous_end_datetime": data_dict["meta"]["previous_end_datetime"]
                }

                df = pd.DataFrame([flat_data])
                ART = df["value"][0]

                if ART is not None:
                    ART_Min = ART / 3600
                else:
                    ART_Min = 0  # or None, or skip, depending on your logic


                return ART_Min  # Return the extracted value

            except KeyError:
                print("Unexpected response format:", response.json())
                return None

        elif response.status_code == 429:  # Rate limited
            wait_time = 2 ** retries  # Exponential backoff (2, 4, 8, 16, etc.)
            print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
            retries += 1
        else:
            print(f"API request failed: {response.status_code} - {response.text}")
            return None  # Return None if the API call fails

    print("Max retries reached. API request failed.")
    return None


StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 19, Finished, Available, Finished)

In [13]:
def call_api_for_one_touch(userid, start_time, end_time, channel, client, authorization):
    """Fetches the One-Touch Tickets metric from Gorgias API."""
    api_url = f"https://{client}.gorgias.com/api/stats/total-one-touch-tickets"
    payload = {
        "filters": {
            "period": {
                "start_datetime": start_time,
                "end_datetime": end_time
            },
            "agents": [userid],
            "channels": [channel]
        }
    }
    headers = {
        "accept": "application/json",
        "authorization": f"Basic {authorization}"
    }
    
    retries = 0
    while retries < 5:
        response = requests.post(api_url, json=payload, headers=headers)
        
        if response.status_code == 429:
            retry_after = int(response.headers.get("Retry-After", 2 ** retries))
            print(f"Rate limited. Retrying in {retry_after} seconds...")
            time.sleep(retry_after)
            retries += 1
        elif response.status_code in [200, 201]:
            data = response.json()
            data_info = data.get("data", {}).get("data", {})
            meta_info = data.get("meta", {})
            flattened_data = {**data_info, **meta_info}
            df = pd.DataFrame([flattened_data])
            return df["value"][0]  # Extract the metric value
        else:
            print(f"API request failed in call_api_for_one_touch function: {response.status_code}")
            return None
    
    print("Max retries reached for call_api_for_one_touch.")
    return None


StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 20, Finished, Available, Finished)

In [14]:
# Convert JSON string back to dictionary
json_dict = json.loads(json_str)
# Create credentials from JSON dictionary
creds = Credentials.from_service_account_info(json_dict, scopes=["https://www.googleapis.com/auth/spreadsheets"])

# Authenticate with Google Sheets
gc = gspread.authorize(creds)

# Open Google Sheet (Replace with your Google Sheet ID)
SHEET_ID = "101FEGEv6lu8mPcozqbimKzWRSH0e103ZApjTaRpHL2w"
sheet = gc.open_by_key(SHEET_ID).sheet1  # Read the first sheet

# Read data into Pandas DataFrame
data = sheet.get_all_records()
df_clients = pd.DataFrame(data)

print(df_clients)  # Output the DataFrame

# Loop through DataFrame rows

df_updated = pd.DataFrame(columns=["Client","ID", "Name", "Email", "Month_start_date","Ticket", "Messages", "FRT", "Customer_Satisfaction-Messages_Sent", "Customer_Satisfaction-Response_Rate_in_Percentage",
                        "Customer_Satisfaction-Rating", "One_Touch_in_Percentage", "ART", "Channel"])

for index, row in df_clients.iterrows():
    print(f"Processing ID: {row['Client']}")
    client = row['Client']
    channel = row["Channel"]
    authorization = row["Authentication"]
    filtered_df = process_users(client, authorization)
     
    
    updated_result = process_data_for_monthly_dashboard(filtered_df, channel, client, authorization)

    if updated_result:
        # Convert updated_result to DataFrame
        df_new = pd.DataFrame(updated_result)

        # Select only relevant columns
        df_new = df_new[df_updated.columns]

        # Append the new data
        df_updated = pd.concat([df_updated, df_new], ignore_index=True)

StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 21, Finished, Available, Finished)

      Client                                     Authentication Channel
0  attngrace  YXR0bmdyYWNlQGdldHdycmsuY29tOjBjMTlmNDJjYTYzNj...   email
1  apupabove  Z3JvY2VyeXB1cEBnZXR3cnJrLmNvbTowMDFkYmJjY2IyNT...   email
2  apupabove  Z3JvY2VyeXB1cEBnZXR3cnJrLmNvbTowMDFkYmJjY2IyNT...    chat
Processing ID: attngrace
start_time, 2025-03-01T00:00:00-05:00
start_time, 2025-02-01T00:00:00-05:00
start_time, 2025-01-01T00:00:00-05:00
start_time, 2024-12-01T00:00:00-05:00
start_time, 2024-11-01T00:00:00-05:00
start_time, 2024-10-01T00:00:00-05:00
start_time, 2024-09-01T00:00:00-05:00
Rate limit exceeded. Retrying in 7 seconds...
start_time, 2024-08-01T00:00:00-05:00
start_time, 2024-07-01T00:00:00-05:00
start_time, 2024-06-01T00:00:00-05:00
start_time, 2024-05-01T00:00:00-05:00
start_time, 2024-04-01T00:00:00-05:00
start_time, 2024-03-01T00:00:00-05:00
start_time, 2024-02-01T00:00:00-05:00
start_time, 2024-01-01T00:00:00-05:00
start_time, 2023-12-01T00:00:00-05:00
start_time, 2023-11-01T00:00:00-0

In [15]:
df_updated.head(100)

StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 22, Finished, Available, Finished)

,Client,ID,Name,Email,Month_start_date,Ticket,Messages,FRT,Customer_Satisfaction-Messages_Sent,Customer_Satisfaction-Response_Rate_in_Percentage,Customer_Satisfaction-Rating,One_Touch_in_Percentage,ART,Channel
0,attngrace,442377384,Cherry N,cherri.northcutt@getwrrk.com,2025-03-01T00:00:00-05:00,657,702,30,537,21,4,6,44.427222,email
1,attngrace,442377384,Cherry N,cherri.northcutt@getwrrk.com,2025-02-01T00:00:00-05:00,571,619,31,470,22,4,8,48.896944,email
2,attngrace,442377384,Cherry N,cherri.northcutt@getwrrk.com,2025-01-01T00:00:00-05:00,677,751,33,562,21,4,7,55.046389,email
3,attngrace,442377384,Cherry N,cherri.northcutt@getwrrk.com,2024-12-01T00:00:00-05:00,572,646,53,499,23,4,6,76.788611,email
4,attngrace,442377384,Cherry N,cherri.northcutt@getwrrk.com,2024-11-01T00:00:00-05:00,471,503,41,394,21,4,7,59.290833,email
5,attngrace,442377384,Cherry N,cherri.northcutt@getwrrk.com,2024-10-01T00:00:00-05:00,618,702,23,508,23,4,9,42.378611,email
6,attngrace,442377384,Cherry N,cherri.northcutt@getwrrk.com,2024-09-01T00:00:00-05:00,609,632,25,488,24,4,8,47.970000,email
7,attngrace,442377384,Cherry N,cherri.northcutt@getwrrk.com,2024-08-01T00:00:00-05:00,644,665,17,499,31,4,11,23.090000,email
8,attngrace,442377384,Cherry N,cherri.northcutt@getwrrk.com,2024-07-01T00:00:00-05:00,703,771,22,564,23,4,12,37.403333,email
9,attngrace,442377384,Cherry N,cherri.northcutt@getwrrk.com,2024-06-01T00:00:00-05:00,671,594,27,418,28,4,21,53.656111,email


In [ ]:
df_updated.info()

StatementMeta(, 9dccb349-9d41-43e2-a860-0dd437f9e85f, 54, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 16 columns):
 #   Column                                             Non-Null Count  Dtype                    
---  ------                                             --------------  -----                    
 0   Client                                             18 non-null     object                   
 1   ID                                                 18 non-null     object                   
 2   Name                                               18 non-null     object                   
 3   Email                                              18 non-null     object                   
 4   Month_start_date                                   18 non-null     datetime64[ns, UTC-05:00]
 5   Ticket                                             18 non-null     object                   
 6   Messages                                           18 non-null     object                   
 7   FRT       

In [16]:
# Save as CSV
df_updated.to_csv("/lakehouse/default/Files/df_output.csv", index=False1

StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 23, Finished, Available, Finished)

In [17]:
# Add new timestamp columns
current_time = datetime.now()
df_updated["Created_Datetime"] = current_time
df_updated["Last_updated_time"] = current_time

StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 24, Finished, Available, Finished)

In [1]:
df_updated.head(100)

StatementMeta(, c6db59bc-08f7-420e-9618-d150b934a11d, 3, Finished, Available, Finished)

NameError: name 'df_updated' is not defined

In [17]:
len(df_updated)

StatementMeta(, 9dccb349-9d41-43e2-a860-0dd437f9e85f, 46, Finished, Available, Finished)

18

In [18]:
spark_df_updated = spark.createDataFrame(df_updated)

StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 25, Finished, Available, Finished)

In [ ]:
from pyspark.sql.functions import to_date
spark_df_updated = spark_df_updated.withColumn("Month_start_date", to_date("Month_start_date"))

StatementMeta(, 9dccb349-9d41-43e2-a860-0dd437f9e85f, 61, Finished, Available, Finished)

In [31]:
spark_df_updated.show()

StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 38, Finished, Available, Finished)

+---------+---------+--------+--------------------+--------------------+------+--------+---+-----------------------------------+-------------------------------------------------+----------------------------+-----------------------+------------------+-------+--------------------+--------------------+
|   Client|       ID|    Name|               Email|    Month_start_date|Ticket|Messages|FRT|Customer_Satisfaction-Messages_Sent|Customer_Satisfaction-Response_Rate_in_Percentage|Customer_Satisfaction-Rating|One_Touch_in_Percentage|               ART|Channel|    Created_Datetime|   Last_updated_time|
+---------+---------+--------+--------------------+--------------------+------+--------+---+-----------------------------------+-------------------------------------------------+----------------------------+-----------------------+------------------+-------+--------------------+--------------------+
|attngrace|442377384|Cherry N|cherri.northcutt@...|2025-03-01T00:00:...|   657|     702| 30|     

In [22]:
#Create the table - to be run only once
#spark_df_updated.write.format("delta").save("Tables/Monthly_Dashboard")
df_original_dashboard.write.format("delta").save("Tables/Monthly_Dashboard_New1")
#spark_df_updated.write.mode("overwrite").saveAsTable("Monthly_Dashboard")

StatementMeta(, c6db59bc-08f7-420e-9618-d150b934a11d, 24, Finished, Available, Finished)

In [17]:
# Read data from ticket table to get distinct user_id and assignee_user_email  
df_original_dashboard = spark.sql("SELECT * FROM Data_from_Bigquery.monthly_dashboard")


StatementMeta(, c6db59bc-08f7-420e-9618-d150b934a11d, 19, Finished, Available, Finished)

In [20]:
df_original_dashboard.show()

StatementMeta(, c6db59bc-08f7-420e-9618-d150b934a11d, 22, Finished, Available, Finished)

+---------+---------+------------------+--------------------+----------------+------+--------+---+-----------------------------------+-------------------------------------------------+----------------------------+-----------------------+------------------+-------+--------------------+--------------------+
|   Client|       ID|              Name|               Email|Month_start_date|Ticket|Messages|FRT|Customer_Satisfaction-Messages_Sent|Customer_Satisfaction-Response_Rate_in_Percentage|Customer_Satisfaction-Rating|One_Touch_in_Percentage|               ART|Channel|    Created_Datetime|   Last_updated_time|
+---------+---------+------------------+--------------------+----------------+------+--------+---+-----------------------------------+-------------------------------------------------+----------------------------+-----------------------+------------------+-------+--------------------+--------------------+
|attngrace|442377384|          Cherry N|cherri.northcutt@...|      2025-03-01| 

In [19]:
from pyspark.sql.functions import to_date
df_original_dashboard = df_original_dashboard.withColumn("Month_start_date", to_date("Month_start_date"))

StatementMeta(, c6db59bc-08f7-420e-9618-d150b934a11d, 21, Finished, Available, Finished)

In [23]:
# Rename the column
df_original_dashboard_renamed = df_original_dashboard.withColumnRenamed("One_Touch_in_Percentage", "One_Touch")



StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 30, Finished, Available, Finished)

In [29]:
# Overwrite the table with the updated schema (optional)
df_original_dashboard.write.format("delta").save("Tables/Monthly_Dashboard")



StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, 36, Finished, Available, Finished)

In [ ]:
# Join DataFrames based on ID and Month_start_date
joined__common_record_df = spark_df_updated.alias("update").join(df_original_dashboard.alias("existing"),
                                        (spark_df_updated.ID == df_original_dashboard.ID) & (spark_df_updated.Month_start_date == df_original_dashboard.Month_start_date),
                                        "left").select(
                                        #spark_df_updated["*"],  # Select all columns from the updated DataFrame
                                        df_original_dashboard["ID"],  # Replace with actual column names you need
                                        df_original_dashboard["Month_start_date"],
                                        df_original_dashboard["Created_Datetime"])

StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, -1, Cancelled, , Cancelled)

In [ ]:
joined__common_record_df.show()

StatementMeta(, a7ae82d5-8975-454e-914f-315ccd1ea216, -1, Cancelled, , Cancelled)

In [ ]:
joined__common_record_df = joined__common_record_df.withColumnRenamed("Created_Datetime", "Created_Datetime_original")

In [26]:
joined__common_record_df.show()

StatementMeta(, 910e88b2-9b58-4840-895d-b31044aec4d5, 34, Finished, Available, Finished)

+---------+--------------------+-------------------------+
|       ID|    Month_start_date|Created_Datetime_original|
+---------+--------------------+-------------------------+
|442377384|2025-03-01T00:00:...|     2025-04-09 13:10:...|
|442377384|2025-02-01T00:00:...|     2025-04-09 13:10:...|
|442377384|2025-01-01T00:00:...|     2025-04-09 13:10:...|
|442377384|2024-12-01T00:00:...|     2025-04-09 13:10:...|
|442377384|2024-11-01T00:00:...|     2025-04-09 13:10:...|
|442377666|2025-03-01T00:00:...|     2025-04-09 13:10:...|
|442377666|2025-01-01T00:00:...|     2025-04-09 13:10:...|
|442377666|2024-12-01T00:00:...|     2025-04-09 13:10:...|
|442377666|2024-11-01T00:00:...|     2025-04-09 13:10:...|
|733976535|2025-03-01T00:00:...|     2025-04-09 13:10:...|
|442377687|2025-03-01T00:00:...|     2025-04-09 13:10:...|
|537816857|2024-12-01T00:00:...|     2025-04-09 13:10:...|
|537816857|2024-11-01T00:00:...|     2025-04-09 13:10:...|
|699488902|2025-03-01T00:00:...|     2025-04-09 13:10:..

In [27]:

# Perform join on 'ID' and 'Month_start_date'
final_df = joined__common_record_df.join(spark_df_updated, on=["ID", "Month_start_date"], how="right")



StatementMeta(, 910e88b2-9b58-4840-895d-b31044aec4d5, 35, Finished, Available, Finished)

In [28]:
final_df.show()

StatementMeta(, 910e88b2-9b58-4840-895d-b31044aec4d5, 36, Finished, Available, Finished)

+---------+--------------------+-------------------------+---------+------------------+--------------------+------+--------+---+-----------------------------------+-------------------------------------------------+----------------------------+-----------------------+------------------+-------+--------------------+--------------------+
|       ID|    Month_start_date|Created_Datetime_original|   Client|              Name|               Email|Ticket|Messages|FRT|Customer_Satisfaction-Messages_Sent|Customer_Satisfaction-Response_Rate_in_Percentage|Customer_Satisfaction-Rating|One_Touch_in_Percentage|               ART|Channel|    Created_Datetime|   Last_updated_time|
+---------+--------------------+-------------------------+---------+------------------+--------------------+------+--------+---+-----------------------------------+-------------------------------------------------+----------------------------+-----------------------+------------------+-------+--------------------+-----------

In [29]:
from pyspark.sql.functions import when, col
# Update Created_Datetime with Created_Datetime_original's value if Created_Datetime_original is not null
final_df = final_df.withColumn(
    "Created_Datetime",  # The column to update
    when(col("Created_Datetime_original").isNotNull(), col("Created_Datetime_original")).otherwise(col("Created_Datetime"))  
)

StatementMeta(, 910e88b2-9b58-4840-895d-b31044aec4d5, 37, Finished, Available, Finished)

In [30]:
# Assuming df is the Spark DataFrame and we want to drop the column "col_name"

final_df = final_df.drop("Created_Datetime_original")

# Show the resulting DataFrame after dropping the column
final_df.show()


StatementMeta(, 910e88b2-9b58-4840-895d-b31044aec4d5, 38, Finished, Available, Finished)

+---------+--------------------+---------+------------------+--------------------+------+--------+---+-----------------------------------+-------------------------------------------------+----------------------------+-----------------------+------------------+-------+--------------------+--------------------+
|       ID|    Month_start_date|   Client|              Name|               Email|Ticket|Messages|FRT|Customer_Satisfaction-Messages_Sent|Customer_Satisfaction-Response_Rate_in_Percentage|Customer_Satisfaction-Rating|One_Touch_in_Percentage|               ART|Channel|    Created_Datetime|   Last_updated_time|
+---------+--------------------+---------+------------------+--------------------+------+--------+---+-----------------------------------+-------------------------------------------------+----------------------------+-----------------------+------------------+-------+--------------------+--------------------+
|442377384|2025-03-01T00:00:...|attngrace|          Cherry N|cherri

In [153]:
#final_merged_df.write.mode("overwrite").saveAsTable("Monthly_Dashboard")


StatementMeta(, 910e88b2-9b58-4840-895d-b31044aec4d5, 39, Finished, Available, Finished)

In [20]:
#spark.sql("DELETE FROM Monthly_Dashboard")

StatementMeta(, 910e88b2-9b58-4840-895d-b31044aec4d5, 40, Finished, Available, Finished)

In [31]:
# Perform a full outer join to merge df1 and df2 on the columns ID and Month_start_date
merged_df = df_original_dashboard.alias("df1").join(
    final_df.alias("df2"),
    (col("df1.ID") == col("df2.ID")) & (col("df1.Month_start_date") == col("df2.Month_start_date")),
    how="outer"
)

StatementMeta(, 910e88b2-9b58-4840-895d-b31044aec4d5, 41, Finished, Available, Finished)

In [ ]:
merged_df.show()

StatementMeta(, 910e88b2-9b58-4840-895d-b31044aec4d5, 42, Finished, Available, Finished)

+---------+---------+------------------+--------------------+--------------------+------+--------+---+-----------------------------------+-------------------------------------------------+----------------------------+-----------------------+------------------+-------+--------------------+--------------------+---------+--------------------+---------+------------------+--------------------+------+--------+---+-----------------------------------+-------------------------------------------------+----------------------------+-----------------------+------------------+-------+--------------------+--------------------+
|   Client|       ID|              Name|               Email|    Month_start_date|Ticket|Messages|FRT|Customer_Satisfaction-Messages_Sent|Customer_Satisfaction-Response_Rate_in_Percentage|Customer_Satisfaction-Rating|One_Touch_in_Percentage|               ART|Channel|    Created_Datetime|   Last_updated_time|       ID|    Month_start_date|   Client|              Name|          

In [ ]:
df_updated = pd.DataFrame(columns=["Client","ID", "Name", "Email", "Month_start_date","Ticket", "Messages", "FRT", "Customer_Satisfaction-Messages_Sent", "Customer_Satisfaction-Response_Rate_in_Percentage",
                        "Customer_Satisfaction-Rating", "One_Touch_in_Percentage", "ART", "Channel"])

StatementMeta(, 910e88b2-9b58-4840-895d-b31044aec4d5, 43, Finished, Available, Finished)

In [32]:
# Select the columns from df2 if they exist, else fallback to df1's columns
final_merged_df = merged_df.select(
    col("df2.Client").alias("Client"),
    col("df2.ID").alias("ID"),
    col("df2.Name").alias("Name"),
    col("df2.Email").alias("Email"),
    col("df2.Month_start_date").alias("Month_start_date"),
    col("df2.Ticket").alias("Ticket"),
    col("df2.Messages").alias("Messages"),
    col("df2.FRT").alias("FRT"),
    col("df2.One_Touch_in_Percentage").alias("One_Touch_in_Percentage"),
    col("df2.ART").alias("ART"),
    col("df2.Channel").alias("Channel"),
    col("df2.Last_updated_time").alias("Last_updated_time"),
    col("df2.Customer_Satisfaction-Messages_Sent").alias("Customer_Satisfaction-Messages_Sent"),
    col("df2.Customer_Satisfaction-Response_Rate_in_Percentage").alias("Customer_Satisfaction-Response_Rate_in_Percentage"),
    col("df2.Customer_Satisfaction-Rating").alias("Customer_Satisfaction-Rating"),
    # Add other columns from df1 as a fallback if df2 is null (for cases where no match exists in df2)
    col("df1.Created_Datetime").alias("Created_Datetime")
)



StatementMeta(, 910e88b2-9b58-4840-895d-b31044aec4d5, 44, Finished, Available, Finished)

In [33]:
final_merged_df.show()

StatementMeta(, 910e88b2-9b58-4840-895d-b31044aec4d5, 45, Finished, Available, Finished)

+---------+---------+------------------+--------------------+--------------------+------+--------+---+-----------------------+------------------+-------+--------------------+-----------------------------------+-------------------------------------------------+----------------------------+--------------------+
|   Client|       ID|              Name|               Email|    Month_start_date|Ticket|Messages|FRT|One_Touch_in_Percentage|               ART|Channel|   Last_updated_time|Customer_Satisfaction-Messages_Sent|Customer_Satisfaction-Response_Rate_in_Percentage|Customer_Satisfaction-Rating|    Created_Datetime|
+---------+---------+------------------+--------------------+--------------------+------+--------+---+-----------------------+------------------+-------+--------------------+-----------------------------------+-------------------------------------------------+----------------------------+--------------------+
|attngrace|442377384|          Cherry N|cherri.northcutt@...|2024-1

In [34]:
#Overwrite the data
final_merged_df.write.mode("overwrite").saveAsTable("Monthly_Dashboard")

StatementMeta(, 910e88b2-9b58-4840-895d-b31044aec4d5, 46, Finished, Available, Finished)